# 2. Stateful Conversations with Memory

To build intelligent chatbots, an LLM needs to remember past interactions. This notebook explores LangChain's `Memory` modules, which provide the context necessary for stateful conversations. We cover various strategies for managing conversation history, from simple buffers that store recent messages (`ConversationBufferWindowMemory`) to more advanced techniques that summarize the entire dialogue (`ConversationSummaryBufferMemory`) to manage token limits effectively.

## ConversationBufferMemory

In [13]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [42]:
llm_model = "gpt-3.5-turbo"

In [43]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [44]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [45]:
conversation.predict(input="Hi, my name is Nabeel Shan")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Nabeel Shan
AI:

> Finished chain.


"Hello Nabeel Shan! It's nice to meet you. How can I assist you today?"

In [46]:
conversation.predict(input="What is the meaning of my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Nabeel Shan
AI: Hello Nabeel Shan! It's nice to meet you. How can I assist you today?
Human: What is the meaning of my name?
AI:

> Finished chain.


'The name Nabeel is of Arabic origin and means "noble" or "noble-minded." Shan is a common surname in various cultures and can have different meanings depending on the specific cultural context.'

In [47]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Nabeel Shan
AI: Hello Nabeel Shan! It's nice to meet you. How can I assist you today?
Human: What is the meaning of my name?
AI: The name Nabeel is of Arabic origin and means "noble" or "noble-minded." Shan is a common surname in various cultures and can have different meanings depending on the specific cultural context.
Human: What is 1+1?
AI:

> Finished chain.


"1+1 equals 2. It's a simple addition problem with a straightforward answer. Is there anything else you'd like to know or discuss?"

In [48]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Nabeel Shan
AI: Hello Nabeel Shan! It's nice to meet you. How can I assist you today?
Human: What is the meaning of my name?
AI: The name Nabeel is of Arabic origin and means "noble" or "noble-minded." Shan is a common surname in various cultures and can have different meanings depending on the specific cultural context.
Human: What is 1+1?
AI: 1+1 equals 2. It's a simple addition problem with a straightforward answer. Is there anything else you'd like to know or discuss?
Human: What is my name?
AI:

> Finished chain.


'Your name is Nabeel Shan.'

In [49]:
print(memory.buffer)

Human: Hi, my name is Nabeel Shan
AI: Hello Nabeel Shan! It's nice to meet you. How can I assist you today?
Human: What is the meaning of my name?
AI: The name Nabeel is of Arabic origin and means "noble" or "noble-minded." Shan is a common surname in various cultures and can have different meanings depending on the specific cultural context.
Human: What is 1+1?
AI: 1+1 equals 2. It's a simple addition problem with a straightforward answer. Is there anything else you'd like to know or discuss?
Human: What is my name?
AI: Your name is Nabeel Shan.


In [50]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is Nabeel Shan\nAI: Hello Nabeel Shan! It\'s nice to meet you. How can I assist you today?\nHuman: What is the meaning of my name?\nAI: The name Nabeel is of Arabic origin and means "noble" or "noble-minded." Shan is a common surname in various cultures and can have different meanings depending on the specific cultural context.\nHuman: What is 1+1?\nAI: 1+1 equals 2. It\'s a simple addition problem with a straightforward answer. Is there anything else you\'d like to know or discuss?\nHuman: What is my name?\nAI: Your name is Nabeel Shan.'}

In [51]:
memory = ConversationBufferMemory()

In [52]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [53]:
print(memory.buffer)

Human: Hi
AI: What's up


In [54]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [55]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [56]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

In [57]:
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool


## ConversationBufferWindowMemory

In [58]:
from langchain.memory import ConversationBufferWindowMemory

In [59]:
memory = ConversationBufferWindowMemory(k=1)               

In [60]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [61]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [62]:
print(memory.buffer)

[HumanMessage(content='Hi', additional_kwargs={}, example=False), AIMessage(content="What's up", additional_kwargs={}, example=False), HumanMessage(content='Not much, just hanging', additional_kwargs={}, example=False), AIMessage(content='Cool', additional_kwargs={}, example=False)]


In [63]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [64]:
conversation.predict(input="Hi, my name is Nabeel Shan")

"Hello Nabeel Shan! It's nice to meet you. How can I assist you today?"

In [65]:
conversation.predict(input="What is 1+1?")

'1 + 1 equals 2. Is there anything else you would like to know?'

In [66]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

In [67]:
#!pip install tiktoken

In [68]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [69]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [70]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [71]:
from langchain.memory import ConversationSummaryBufferMemory

In [72]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [73]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.'}

In [74]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [75]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or case studies would be beneficial to showcase the real-world impact of LangChain.'

In [76]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.\nHuman: What would be a good demo to show?\nAI: For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or case studies would be beneficial to showcase the real-world impact of LangChain.'}

In [77]:
conversation.predict(input="What i should do now?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI: For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or c

'To prepare for the meeting with the product team, you could gather any relevant data or statistics on the LangChain project, review the latest features and updates, and ensure that the demo is running smoothly. It may also be helpful to brainstorm potential questions or concerns that the product team may have and prepare responses in advance. Additionally, you could reach out to any team members or stakeholders for input or feedback to ensure that the demo effectively showcases the capabilities of LangChain.'

In [78]:
memory.load_memory_variables({})

{'history': 'System: The human and AI discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool. For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal, including a live demonstration of how LangChain streamlines language translation processes and improves efficiency. Additionally, highlighting recent success stories or case studies would be beneficial. The human asks what they should do next.\nAI: To prepare for the meeting with the product team, you could gather any relevant data or statistics on the LangChain project, review the latest features and updates, and ensure that the demo is running smoothly. It may also be helpful to brainstorm potential questions or concerns that the product team may have and prepare responses i

In [79]:
conversation.predict(input="ok, i am going to collect data")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool. For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal, including a live demonstration of how LangChain streamlines language translation processes and improves efficiency. Additionally, highlighting recent success stories or case studies would be beneficial. The human asks what they should do next.
AI: To prepare for the meetin

'Great idea! Gathering data will help you provide a comprehensive overview of the LangChain project during the meeting with the product team. Make sure to focus on key metrics, success stories, and any relevant case studies to showcase the impact of LangChain. If you need any assistance or have any questions while collecting data, feel free to ask for help. Good luck with your preparations!'

In [80]:
memory.load_memory_variables({})

{'history': "System: The human and AI discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool. For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal, including a live demonstration of how LangChain streamlines language translation processes and improves efficiency. Additionally, highlighting recent success stories or case studies would be beneficial. The human asks what they should do next. The AI suggests preparing for the meeting with the product team by gathering relevant data, reviewing features and updates, ensuring the demo runs smoothly, brainstorming potential questions, and seeking input from team members or stakeholders to effectively showcase LangChain's capabilities.\nHuman: ok, i am going to collect data\nAI: G